# Sistema de archivos e intérprete de comandos

Vamos a ver como hace <b>JOS</b> para manejarse con el <b>file system</b>. Para esto el <b>kernel</b> levanta un proceso que es el que se encargara de <b>leer</b> y <b>escribir</b> en el disco. Este proceso se lanza en la funcion <b>i386_init</b>:

```c
void i386_init(void) {
    // ...

    // Start fs.
    ENV_CREATE(fs_fs, ENV_TYPE_FS);

    // ...
}
```

Notemos que el tipo de este proceso es <b>ENV_TYPE_FS</b>. Es decir no es de tipo <b>ENV_TYPE_USER</b>. Esto es para que los demas procesos puedan encontrar facilmente este, preguntando el tipo. 

## Modelado de un archivo:

Aunque en realidad es un <b>file node</b>, representa la metadata del file que realmente esta en memoria.

```c
struct File {
	char f_name[MAXNAMELEN];	// filename
	off_t f_size;			// file size in bytes
	uint32_t f_type;		// file type

	// Block pointers.
	// A block is allocated iff its value is != 0.
	uint32_t f_direct[NDIRECT];	// direct blocks
	uint32_t f_indirect;		// indirect block

	// Pad out to 256 bytes; must do arithmetic in case we're compiling
	// fsformat on a 64-bit machine.
	uint8_t f_pad[256 - MAXNAMELEN - 8 - 4*NDIRECT - 4];
} __attribute__((packed));	// required only on some 64-bit machines
```